# Skorch Basic usage

*`skorch`* is designed to maximize interoperability between `sklearn` and `pytorch`. The aim is to keep 99% of the flexibility of `pytorch` while being able to leverage most features of `sklearn`. Below, we show the basic usage of `skorch` and how it can be combined with `sklearn`.

<table align="left"><td>
<a target="_blank" href="https://colab.research.google.com/github/skorch-dev/skorch/blob/master/notebooks/Basic_Usage.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>  
</td><td>
<a target="_blank" href="https://github.com/skorch-dev/skorch/blob/master/notebooks/Basic_Usage.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a></td></table>

This notebook shows you how to use the basic functionality of `skorch`.

### Table of contents

* [Definition of the pytorch module](#Definition-of-the-pytorch-module)
* [Training a classifier](#Training-a-classifier-and-making-predictions)
  * [Dataset](#A-toy-binary-classification-task)
  * [pytorch module](#Definition-of-the-pytorch-classification-module)
  * [Model training](#Defining-and-training-the-neural-net-classifier)
  * [Inference](#Making-predictions,-classification)
* [Training a regressor](#Training-a-regressor)
  * [Dataset](#A-toy-regression-task)
  * [pytorch module](#Definition-of-the-pytorch-regression-module)
  * [Model training](#Defining-and-training-the-neural-net-regressor)
  * [Inference](#Making-predictions,-regression)
* [Saving and loading a model](#Saving-and-loading-a-model)
  * [Whole model](#Saving-the-whole-model)
  * [Only parameters](#Saving-only-the-model-parameters)
* [Usage with an sklearn Pipeline](#Usage-with-an-sklearn-Pipeline)
* [Callbacks](#Callbacks)
* [Grid search](#Usage-with-sklearn-GridSearchCV)
  * [Special prefixes](#Special-prefixes)
  * [Performing a grid search](#Performing-a-grid-search)

In [21]:
#! [ ! -z "$COLAB_GPU" ] && pip install torch skorch

In [22]:
import torch
from torch import nn
import torch.nn.functional as F

In [23]:
torch.manual_seed(0)
torch.cuda.manual_seed(0)

## Training a classifier and making predictions

### A toy binary classification task

We load a toy classification task from `sklearn`.

In [24]:
import numpy as np
from sklearn.datasets import make_classification

In [25]:
X, y = make_classification(1000, 20, n_informative=10, random_state=0)
X, y = X.astype(np.float32), y.astype(np.int64)

In [26]:
X.shape, y.shape, y.mean()

((1000, 20), (1000,), 0.5)

### Definition of the `pytorch` classification `module`

We define a vanilla neural network with two hidden layers. The output layer should have 2 output units since there are two classes. In addition, it should have a softmax nonlinearity, because later, when calling `predict_proba`, the output from the `forward` call will be used.

In [27]:
class ClassifierModule(nn.Module):
    def __init__(
            self,
            num_units=10,
            nonlin=F.relu,
            dropout=0.5,
    ):
        super(ClassifierModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin
        self.dropout = dropout

        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(dropout)
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 2)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = F.relu(self.dense1(X))
        X = F.softmax(self.output(X), dim=-1)
        return X

### Defining and training the neural net classifier

We use `NeuralNetClassifier` because we're dealing with a classifcation task. The first argument should be the `pytorch module`. As additional arguments, we pass the number of epochs and the learning rate (`lr`), but those are optional.

*Note*: To use the CUDA backend, pass `device='cuda'` as an additional argument.

In [28]:
from skorch import NeuralNetClassifier

In [29]:
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
#     device='cuda',  # uncomment this to train with CUDA
)

As in `sklearn`, we call `fit` passing the input data `X` and the targets `y`. By default, `NeuralNetClassifier` makes a `StratifiedKFold` split on the data (80/20) to track the validation loss. This is shown, as well as the train loss and the accuracy on the validation set.

In [30]:
net.fit(X, y)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6905       0.6150        0.6749  0.0145
      2        0.6740       0.6200        0.6668  0.0150
      3        0.6594       0.6750        0.6554  0.0147
      4        0.6482       0.6900        0.6452  0.0156
      5        0.6423       0.7050        0.6333  0.0132
      6        0.6231       0.7000        0.6188  0.0153
      7        0.6081       0.7100        0.6064  0.0159
      8        0.6003       0.7000        0.5940  0.0152
      9        0.5937       0.7250        0.5836  0.0139
     10        0.5830       0.7150        0.5725  0.0149
     11        0.5686       0.7100        0.5660  0.0141
     12        0.5701       0.7150        0.5577  0.0168
     13        0.5751       0.7200        0.5499  0.0149
     14        0.5662       0.7250        0.5438  0.0156
     15        0.5422       0.7250        0.5354  0.0150
     16        0.5363       0.7

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=20, out_features=10, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (dense1): Linear(in_features=10, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=2, bias=True)
  ),
)

In [31]:
class subClass(ClassifierModule):
    _hidden_neuron_cls = 5
    
class subsubClass(ClassifierModule):
    _hidden_neuron_cls = 6
    

In [32]:
net2 = NeuralNetClassifier(
    subClass,
    max_epochs=20,
    lr=0.1,
#     device='cuda',  # uncomment this to train with CUDA
)

net3 = NeuralNetClassifier(
    subsubClass,
    max_epochs=20,
    lr=0.1,
#     device='cuda',  # uncomment this to train with CUDA
)

In [33]:
net2.fit(X, y)
net3.fit(X, y)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6933       0.5700        0.6852  0.0145
      2        0.6855       0.6200        0.6736  0.0174
      3        0.6777       0.6200        0.6663  0.0134
      4        0.6603       0.6450        0.6597  0.0159
      5        0.6560       0.6650        0.6524  0.0159
      6        0.6534       0.6600        0.6465  0.0159
      7        0.6490       0.6650        0.6393  0.0164
      8        0.6305       0.6700        0.6305  0.0146
      9        0.6270       0.6800        0.6202  0.0135
     10        0.6222       0.6750        0.6114  0.0697
     11        0.6110       0.6800        0.6038  0.0150
     12        0.6006       0.7000        0.5973  0.0147
     13        0.6030       0.7000        0.5921  0.0166
     14        0.5913       0.7000        0.5848  0.0161
     15        0.5791       0.7150        0.5795  0.0154
     16        0.5694       0.7

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=subsubClass(
    (dense0): Linear(in_features=20, out_features=10, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (dense1): Linear(in_features=10, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=2, bias=True)
  ),
)

Also, as in `sklearn`, you may call `predict` or `predict_proba` on the fitted model.

### Making predictions, classification

In [34]:
y_pred = net.predict(X[:5])
y_pred

array([0, 0, 0, 0, 0])

In [35]:
y_proba = net.predict_proba(X[:5])
y_proba

array([[0.5603605 , 0.4396395 ],
       [0.782588  , 0.21741197],
       [0.6924924 , 0.3075076 ],
       [0.8895971 , 0.1104029 ],
       [0.70746267, 0.2925373 ]], dtype=float32)

## Training a regressor

### A toy regression task

In [36]:
from sklearn.datasets import make_regression

In [37]:
X_regr, y_regr = make_regression(1000, 20, n_informative=10, random_state=0)
X_regr = X_regr.astype(np.float32)
y_regr = y_regr.astype(np.float32) / 100
y_regr = y_regr.reshape(-1, 1)

In [38]:
X_regr.shape, y_regr.shape, y_regr.min(), y_regr.max()

((1000, 20), (1000, 1), -6.4901485, 6.154505)

*Note*: Regression currently requires the target to be 2-dimensional, hence the need to reshape. This should be fixed with an upcoming version of pytorch.

### Definition of the `pytorch` regression `module`

Again, define a vanilla neural network with two hidden layers. The main difference is that the output layer only has one unit and does not apply a softmax nonlinearity.

In [39]:
class RegressorModule(nn.Module):
    def __init__(
            self,
            num_units=10,
            nonlin=F.relu,
    ):
        super(RegressorModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin

        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 1)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = F.relu(self.dense1(X))
        X = self.output(X)
        return X

### Defining and training the neural net regressor

Training a regressor is almost the same as training a classifier. Mainly, we use `NeuralNetRegressor` instead of `NeuralNetClassifier` (this is the same terminology as in `sklearn`).

In [40]:
from skorch import NeuralNetRegressor

In [41]:
net_regr = NeuralNetRegressor(
    RegressorModule,
    max_epochs=20,
    lr=0.1,
#     device='cuda',  # uncomment this to train with CUDA
)

In [42]:
net_regr.fit(X_regr, y_regr)

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        4.3479        3.0639  0.0383
      2        1.7830        0.5272  0.0148
      3        0.2620        0.1964  0.0170
      4        0.1332        0.1704  0.0163
      5        0.1209        0.1355  0.0168
      6        0.2293        0.5582  0.0182
      7        0.3251        0.1058  0.0177
      8        0.0747        0.0565  0.0156
      9        0.0347        0.0419  0.0168
     10        0.0299        0.0309  0.0156
     11        0.0202        0.0358  0.0167
     12        0.0328        0.0310  0.0168
     13        0.0290        0.0599  0.0173
     14        0.0635        0.0513  0.0170
     15        0.0502        0.0602  0.0164
     16        0.0475        0.0221  0.0161
     17        0.0168        0.0288  0.0165
     18        0.0208        0.0132  0.0153
     19        0.0089        0.0196  0.0166
     20        0.0138        0.0109  0.0165


<class 'skorch.regressor.NeuralNetRegressor'>[initialized](
  module_=RegressorModule(
    (dense0): Linear(in_features=20, out_features=10, bias=True)
    (dense1): Linear(in_features=10, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=1, bias=True)
  ),
)

### Making predictions, regression

You may call `predict` or `predict_proba` on the fitted model. For regressions, both methods return the same value.

In [43]:
y_pred = net_regr.predict(X_regr[:5])
y_pred

array([[ 0.8809004 ],
       [-1.4545293 ],
       [-0.73125255],
       [-0.21497256],
       [-0.3453628 ]], dtype=float32)

## Saving and loading a model

Save and load either the whole model by using pickle or just the learned model parameters by calling `save_params` and `load_params`.

### Saving the whole model

In [44]:
import pickle

In [45]:
file_name = '/tmp/mymodel.pkl'

In [46]:
with open(file_name, 'wb') as f:
    pickle.dump(net, f)

In [47]:
with open(file_name, 'rb') as f:
    new_net = pickle.load(f)

### Saving only the model parameters

This only saves and loads the proper `module` parameters, meaning that hyperparameters such as `lr` and `max_epochs` are not saved. Therefore, to load the model, we have to re-initialize it beforehand.

In [48]:
net.save_params(f_params=file_name)  # a file handler also works

In [49]:
# first initialize the model
new_net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
).initialize()

In [50]:
new_net.load_params(file_name)

## Usage with an `sklearn Pipeline`

It is possible to put the `NeuralNetClassifier` inside an `sklearn Pipeline`, as you would with any `sklearn` classifier.

In [51]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [52]:
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('net', net),
])

In [53]:
pipe.fit(X, y)

Re-initializing module.
Re-initializing criterion.
Re-initializing optimizer.
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6861       0.5700        0.6835  0.0134
      2        0.6831       0.5950        0.6811  0.0140
      3        0.6798       0.5900        0.6782  0.0154
      4        0.6766       0.6050        0.6751  0.0149
      5        0.6751       0.6050        0.6715  0.0154
      6        0.6656       0.6450        0.6663  0.0158
      7        0.6641       0.6250        0.6607  0.0146
      8        0.6532       0.6400        0.6540  0.0162
      9        0.6524       0.6500        0.6474  0.0149
     10        0.6399       0.6650        0.6408  0.0140
     11        0.6420       0.6700        0.6342  0.0140
     12        0.6283       0.6650        0.6271  0.0156
     13        0.6204       0.6750        0.6190  0.0150
     14        0.6030       0.6900        0.6110  0.0145
     15   

Pipeline(steps=[('scale', StandardScaler()),
                ('net',
                 <class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=20, out_features=10, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (dense1): Linear(in_features=10, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=2, bias=True)
  ),
))])

In [54]:
y_proba = pipe.predict_proba(X[:5])
y_proba

array([[0.36018655, 0.6398134 ],
       [0.694329  , 0.30567098],
       [0.66123945, 0.3387605 ],
       [0.7088052 , 0.2911948 ],
       [0.6906064 , 0.3093936 ]], dtype=float32)

To save the whole pipeline, including the pytorch module, use `pickle`.

## Callbacks

Adding a new callback to the model is straightforward. Below we show how to add a new callback that determines the area under the ROC (AUC) score.

In [55]:
from skorch.callbacks import EpochScoring

There is a scoring callback in skorch, `EpochScoring`, which we use for this. We have to specify which score to calculate. We have 3 choices:

* Passing a string: This should be a valid `sklearn` metric. For a list of all existing scores, look [here](http://scikit-learn.org/stable/modules/classes.html#sklearn-metrics-metrics).
* Passing `None`: If you implement your own `.score` method on your neural net, passing `scoring=None` will tell `skorch` to use that.
* Passing a function or callable: If we want to define our own scoring function, we pass a function with the signature `func(model, X, y) -> score`, which is then used.

Note that this works exactly the same as scoring in `sklearn` does.

For our case here, since `sklearn` already implements AUC, we just pass the correct string `'roc_auc'`. We should also tell the callback that higher scores are better (to get the correct colors printed below -- by default, lower scores are assumed to be better). Furthermore, we may specify a `name` argument for `EpochScoring`, and whether to use training data (by setting `on_train=True`) or validation data (which is the default).

In [56]:
auc = EpochScoring(scoring='roc_auc', lower_is_better=False)

Finally, we pass the scoring callback to the `callbacks` parameter as a list and then call `fit`. Notice that we get the printed scores and color highlighting for free.

In [57]:
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
    callbacks=[auc],
)

In [58]:
net.fit(X, y)

  epoch    roc_auc    train_loss    valid_acc    valid_loss     dur
-------  ---------  ------------  -----------  ------------  ------
      1     0.7071        0.6925       0.6150        0.6777  0.0146
      2     0.7544        0.6772       0.6450        0.6673  0.0129
      3     0.7713        0.6729       0.6800        0.6572  0.0149
      4     0.7810        0.6666       0.6900        0.6496  0.0136
      5     0.7865        0.6490       0.6850        0.6383  0.0148
      6     0.7930        0.6538       0.6900        0.6301  0.0151
      7     0.7903        0.6335       0.6900        0.6170  0.0152
      8     0.7881        0.6131       0.6900        0.6022  0.0161
      9     0.7891        0.6049       0.7000        0.5900  0.0137
     10     0.7900        0.6038       0.7050        0.5813  0.0152
     11     0.7893        0.5842       0.7050        0.5725  0.0152
     12     0.7905        0.5899       0.7200        0.5665  0.0155
     13     0.7894        0.5654       0.7300   

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=20, out_features=10, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (dense1): Linear(in_features=10, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=2, bias=True)
  ),
)

For information on how to write custom callbacks, have a look at the [Advanced_Usage](https://nbviewer.jupyter.org/github/skorch-dev/skorch/blob/master/notebooks/Advanced_Usage.ipynb) notebook.

## Usage with sklearn `GridSearchCV`

### Special prefixes

The `NeuralNet` class allows to directly access parameters of the `pytorch module` by using the `module__` prefix. So e.g. if you defined the `module` to have a `num_units` parameter, you can set it via the `module__num_units` argument. This is exactly the same logic that allows to access estimator parameters in `sklearn Pipeline`s and `FeatureUnion`s.

This feature is useful in several ways. For one, it allows to set those parameters in the model definition. Furthermore, it allows you to set parameters in an `sklearn GridSearchCV` as shown below.

In addition to the parameters prefixed by `module__`, you may access a couple of other attributes, such as those of the optimizer by using the `optimizer__` prefix (again, see below). All those special prefixes are stored in the `prefixes_` attribute:

In [59]:
print(', '.join(net.prefixes_))

iterator_train, iterator_valid, callbacks, dataset, module, criterion, optimizer


### Performing a grid search

Below we show how to perform a grid search over the learning rate (`lr`), the module's number of hidden units (`module__num_units`), the module's dropout rate (`module__dropout`), and whether the SGD optimizer should use Nesterov momentum or not (`optimizer__nesterov`).

In [60]:
from sklearn.model_selection import GridSearchCV

In [61]:
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
    optimizer__momentum=0.9,
    verbose=0,
    train_split=False,
)

*Note*: We set the verbosity level to zero (`verbose=0`) to prevent too much print output from being shown. Also, we disable the skorch-internal train-validation split (`train_split=False`) because `GridSearchCV` already splits the training data for us. We only have to leave the skorch-internal split enabled for some specific uses, e.g. to perform `EarlyStopping`.

In [62]:
params = {
    'lr': [0.05, 0.1],
    'module__num_units': [10, 20],
    'module__dropout': [0, 0.5],
    'optimizer__nesterov': [False, True],
}

In [63]:
gs = GridSearchCV(net, params, refit=False, cv=3, scoring='accuracy', verbose=2)

In [64]:
gs.fit(X, y)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] END lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=False; total time=   0.2s
[CV] END lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=False; total time=   0.2s
[CV] END lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=False; total time=   0.2s
[CV] END lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=True; total time=   0.2s
[CV] END lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=True; total time=   0.2s
[CV] END lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=True; total time=   0.2s
[CV] END lr=0.05, module__dropout=0, module__num_units=20, optimizer__nesterov=False; total time=   0.2s
[CV] END lr=0.05, module__dropout=0, module__num_units=20, optimizer__nesterov=False; total time=   0.2s
[CV] END lr=0.05, module__dropout=0, module__num_units=20, optimizer__nesterov=False; total time=   0.

GridSearchCV(cv=3,
             estimator=<class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class '__main__.ClassifierModule'>,
),
             param_grid={'lr': [0.05, 0.1], 'module__dropout': [0, 0.5],
                         'module__num_units': [10, 20],
                         'optimizer__nesterov': [False, True]},
             refit=False, scoring='accuracy', verbose=2)

In [65]:
print(gs.best_score_, gs.best_params_)

0.8699897502292712 {'lr': 0.1, 'module__dropout': 0, 'module__num_units': 20, 'optimizer__nesterov': False}


Of course, we could further nest the `NeuralNetClassifier` within an `sklearn Pipeline`, in which case we just prefix the parameter by the name of the net (e.g. `net__module__num_units`).